In [1]:
import tkinter as tk
from tkinter import ttk
import numpy as np
from tkinter import messagebox
import pandas as pd

In [2]:
!pip install tld

In [3]:
!pip install tldextract

In [4]:
import tldextract
import re
from urllib.parse import urlparse
from tld import get_tld, is_tld
import hashlib

In [5]:
def extract_pri_domain(url):
    try:
        res = get_tld(url, as_object = True, fail_silently=False,fix_protocol=True)
        pri_domain= res.parsed_url.netloc
    except :
        pri_domain= None
    return pri_domain

In [6]:
def hash_encode(category):
  hash_object = hashlib.md5(category.encode())
  return int(hash_object.hexdigest(), 16) % (10 ** 8)

In [7]:
def extract_features(url):

    def extract_url_length(url):
        return len(url)

    def extract_count_letters(url):
        num_letters = sum(char.isalpha() for char in url)
        return num_letters

    def extract_count_digits(url):
        num_digits = sum(char.isdigit() for char in url)
        return num_digits

    def extract_count_special_chars(url):
        special_chars = "!@#$%^&*()_+-=[]{};:,.<>/?`~|"
        num_special_chars = sum(char in special_chars for char in url)
        return num_special_chars

    def extract_abnormal_url(url):
        parsed_url = urlparse(url)
        hostname = parsed_url.hostname
        if hostname:
            hostname = str(hostname)
            match = re.search(hostname, url)
            if match:
                return 1
        return 0

    def extract_have_ip_address(url):
        pattern = r'(([01]?\d\d?|2[0-4]\d|25[0-5])\.([01]?\d\d?|2[0-4]\d|25[0-5])\.([01]?\d\d?|2[0-4]\d|25[0-5])\.' \
                  r'([01]?\d\d?|2[0-4]\d|25[0-5])\/)|' \
                  r'(([01]?\d\d?|2[0-4]\d|25[0-5])\.([01]?\d\d?|2[0-4]\d|25[0-5])\.([01]?\d\d?|2[0-4]\d|25[0-5])\.' \
                  r'([01]?\d\d?|2[0-4]\d|25[0-5])\/)|' \
                  r'((0x[0-9a-fA-F]{1,2})\.(0x[0-9a-fA-F]{1,2})\.(0x[0-9a-fA-F]{1,2})\.(0x[0-9a-fA-F]{1,2})\/)' \
                  r'(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}|' \
                  r'([0-9]+(?:\.[0-9]+){3}:[0-9]+)|' \
                  r'((?:(?:\d|[01]?\d\d|2[0-4]\d|25[0-5])\.){3}(?:25[0-5]|2[0-4]\d|[01]?\d\d|\d)(?:\/\d{1,2})?)'

        match = re.search(pattern, url)
        if match:
            return 1
        else:
            return 0

    def extract_secure_http(url):
        scheme = urlparse(url).scheme
        if scheme == 'https':
            return 1
        else:
            return 0

    def extract_has_shortening_service(url):
        pattern = re.compile(r'bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                             r'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                             r'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                             r'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                             r'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                             r'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                             r'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                             r'tr\.im|link\.zip\.net')
        match = pattern.search(url)
        return int(bool(match))

    def extract_root_domain(url):
        extracted = tldextract.extract(url)
        root_domain = extracted.domain
        return root_domain

    def get_url_region(primary_domain):
      ccTLD_to_region = {
      ".ac": "Ascension Island",
      ".ad": "Andorra",
      ".ae": "United Arab Emirates",
      ".af": "Afghanistan",
      ".ag": "Antigua and Barbuda",
      ".ai": "Anguilla",
      ".al": "Albania",
      ".am": "Armenia",
      ".an": "Netherlands Antilles",
      ".ao": "Angola",
      ".aq": "Antarctica",
      ".ar": "Argentina",
      ".as": "American Samoa",
      ".at": "Austria",
      ".au": "Australia",
      ".aw": "Aruba",
      ".ax": "Åland Islands",
      ".az": "Azerbaijan",
      ".ba": "Bosnia and Herzegovina",
      ".bb": "Barbados",
      ".bd": "Bangladesh",
      ".be": "Belgium",
      ".bf": "Burkina Faso",
      ".bg": "Bulgaria",
      ".bh": "Bahrain",
      ".bi": "Burundi",
      ".bj": "Benin",
      ".bm": "Bermuda",
      ".bn": "Brunei Darussalam",
      ".bo": "Bolivia",
      ".br": "Brazil",
      ".bs": "Bahamas",
      ".bt": "Bhutan",
      ".bv": "Bouvet Island",
      ".bw": "Botswana",
      ".by": "Belarus",
      ".bz": "Belize",
      ".ca": "Canada",
      ".cc": "Cocos Islands",
      ".cd": "Democratic Republic of the Congo",
      ".cf": "Central African Republic",
      ".cg": "Republic of the Congo",
      ".ch": "Switzerland",
      ".ci": "Côte d'Ivoire",
      ".ck": "Cook Islands",
      ".cl": "Chile",
      ".cm": "Cameroon",
      ".cn": "China",
      ".co": "Colombia",
      ".cr": "Costa Rica",
      ".cu": "Cuba",
      ".cv": "Cape Verde",
      ".cw": "Curaçao",
      ".cx": "Christmas Island",
      ".cy": "Cyprus",
      ".cz": "Czech Republic",
      ".de": "Germany",
      ".dj": "Djibouti",
      ".dk": "Denmark",
      ".dm": "Dominica",
      ".do": "Dominican Republic",
      ".dz": "Algeria",
      ".ec": "Ecuador",
      ".ee": "Estonia",
      ".eg": "Egypt",
      ".er": "Eritrea",
      ".es": "Spain",
      ".et": "Ethiopia",
      ".eu": "European Union",
      ".fi": "Finland",
      ".fj": "Fiji",
      ".fk": "Falkland Islands",
      ".fm": "Federated States of Micronesia",
      ".fo": "Faroe Islands",
      ".fr": "France",
      ".ga": "Gabon",
      ".gb": "United Kingdom",
      ".gd": "Grenada",
      ".ge": "Georgia",
      ".gf": "French Guiana",
      ".gg": "Guernsey",
      ".gh": "Ghana",
      ".gi": "Gibraltar",
      ".gl": "Greenland",
      ".gm": "Gambia",
      ".gn": "Guinea",
      ".gp": "Guadeloupe",
      ".gq": "Equatorial Guinea",
      ".gr": "Greece",
      ".gs": "South Georgia and the South Sandwich Islands",
      ".gt": "Guatemala",
      ".gu": "Guam",
      ".gw": "Guinea-Bissau",
      ".gy": "Guyana",
      ".hk": "Hong Kong",
      ".hm": "Heard Island and McDonald Islands",
      ".hn": "Honduras",
      ".hr": "Croatia",
      ".ht": "Haiti",
      ".hu": "Hungary",
      ".id": "Indonesia",
      ".ie": "Ireland",
      ".il": "Israel",
      ".im": "Isle of Man",
      ".in": "India",
      ".io": "British Indian Ocean Territory",
      ".iq": "Iraq",
      ".ir": "Iran",
      ".is": "Iceland",
      ".it": "Italy",
      ".je": "Jersey",
      ".jm": "Jamaica",
      ".jo": "Jordan",
      ".jp": "Japan",
      ".ke": "Kenya",
      ".kg": "Kyrgyzstan",
      ".kh": "Cambodia",
      ".ki": "Kiribati",
      ".km": "Comoros",
      ".kn": "Saint Kitts and Nevis",
      ".kp": "Democratic People's Republic of Korea (North Korea)",
      ".kr": "Republic of Korea (South Korea)",
      ".kw": "Kuwait",
      ".ky": "Cayman Islands",
      ".kz": "Kazakhstan",
      ".la": "Laos",
      ".lb": "Lebanon",
      ".lc": "Saint Lucia",
      ".li": "Liechtenstein",
      ".lk": "Sri Lanka",
      ".lr": "Liberia",
      ".ls": "Lesotho",
      ".lt": "Lithuania",
      ".lu": "Luxembourg",
      ".lv": "Latvia",
      ".ly": "Libya",
      ".ma": "Morocco",
      ".mc": "Monaco",
      ".md": "Moldova",
      ".me": "Montenegro",
      ".mf": "Saint Martin (French part)",
      ".mg": "Madagascar",
      ".mh": "Marshall Islands",
      ".mk": "North Macedonia",
      ".ml": "Mali",
      ".mm": "Myanmar",
      ".mn": "Mongolia",
      ".mo": "Macao",
      ".mp": "Northern Mariana Islands",
      ".mq": "Martinique",
      ".mr": "Mauritania",
      ".ms": "Montserrat",
      ".mt": "Malta",
      ".mu": "Mauritius",
      ".mv": "Maldives",
      ".mw": "Malawi",
      ".mx": "Mexico",
      ".my": "Malaysia",
      ".mz": "Mozambique",
      ".na": "Namibia",
      ".nc": "New Caledonia",
      ".ne": "Niger",
      ".nf": "Norfolk Island",
      ".ng": "Nigeria",
      ".ni": "Nicaragua",
      ".nl": "Netherlands",
      ".no": "Norway",
      ".np": "Nepal",
      ".nr": "Nauru",
      ".nu": "Niue",
      ".nz": "New Zealand",
      ".om": "Oman",
      ".pa": "Panama",
      ".pe": "Peru",
      ".pf": "French Polynesia",
      ".pg": "Papua New Guinea",
      ".ph": "Philippines",
      ".pk": "Pakistan",
      ".pl": "Poland",
      ".pm": "Saint Pierre and Miquelon",
      ".pn": "Pitcairn",
      ".pr": "Puerto Rico",
      ".ps": "Palestinian Territory",
      ".pt": "Portugal",
      ".pw": "Palau",
      ".py": "Paraguay",
      ".qa": "Qatar",
      ".re": "Réunion",
      ".ro": "Romania",
      ".rs": "Serbia",
      ".ru": "Russia",
      ".rw": "Rwanda",
      ".sa": "Saudi Arabia",
      ".sb": "Solomon Islands",
      ".sc": "Seychelles",
      ".sd": "Sudan",
      ".se": "Sweden",
      ".sg": "Singapore",
      ".sh": "Saint Helena",
      ".si": "Slovenia",
      ".sj": "Svalbard and Jan Mayen",
      ".sk": "Slovakia",
      ".sl": "Sierra Leone",
      ".sm": "San Marino",
      ".sn": "Senegal",
      ".so": "Somalia",
      ".sr": "Suriname",
      ".ss": "South Sudan",
      ".st": "São Tomé and Príncipe",
      ".sv": "El Salvador",
      ".sx": "Sint Maarten (Dutch part)",
      ".sy": "Syria",
      ".sz": "Eswatini",
      ".tc": "Turks and Caicos Islands",
      ".td": "Chad",
      ".tf": "French Southern Territories",
      ".tg": "Togo",
      ".th": "Thailand",
      ".tj": "Tajikistan",
      ".tk": "Tokelau",
      ".tl": "Timor-Leste",
      ".tm": "Turkmenistan",
      ".tn": "Tunisia",
      ".to": "Tonga",
      ".tr": "Turkey",
      ".tt": "Trinidad and Tobago",
      ".tv": "Tuvalu",
      ".tw": "Taiwan",
      ".tz": "Tanzania",
      ".ua": "Ukraine",
      ".ug": "Uganda",
      ".uk": "United Kingdom",
      ".us": "United States",
      ".uy": "Uruguay",
      ".uz": "Uzbekistan",
      ".va": "Vatican City",
      ".vc": "Saint Vincent and the Grenadines",
      ".ve": "Venezuela",
      ".vg": "British Virgin Islands",
      ".vi": "U.S. Virgin Islands",
      ".vn": "Vietnam",
      ".vu": "Vanuatu",
      ".wf": "Wallis and Futuna",
      ".ws": "Samoa",
      ".ye": "Yemen",
      ".yt": "Mayotte",
      ".za": "South Africa",
      ".zm": "Zambia",
      ".zw": "Zimbabwe"
      }

      for ccTLD in ccTLD_to_region:
          if primary_domain.endswith(ccTLD):
              return ccTLD_to_region[ccTLD]

      return "Global"

    url = url.replace('www.', '')
    url_len = extract_url_length(url)
    letters_count = extract_count_letters(url)
    digits_count = extract_count_digits(url)
    special_chars_count = extract_count_special_chars(url)
    abnormal_url = extract_abnormal_url(url)
    have_ip = extract_have_ip_address(url)
    secure_http = extract_secure_http(url)
    shortened = extract_has_shortening_service(url)
    pri_domain = extract_pri_domain(url)

    root_domain = extract_root_domain(str(pri_domain))
    root_domain = hash_encode(root_domain)

    url_region = get_url_region(str(pri_domain))
    url_region = hash_encode(url_region)

    return pd.DataFrame({
        'url_len': [url_len],
        'letters_count': [letters_count],
        'digits_count': [digits_count],
        'special_chars_count': [special_chars_count],
        'shortened': [shortened],
        'abnormal_url': [abnormal_url],
        'secure_http': [secure_http],
        'have_ip': [have_ip],
        'url_region': [url_region],
        'root_domain': [root_domain]
    })

In [8]:
import pickle

In [9]:
cd D:/Study/SEM 7/SGP/Mal_URL/

D:\Study\SEM 7\SGP\Mal_URL


In [10]:
def action():
    global URL
    URL=url.get()

In [11]:
def make_predict():
    global result
    user_url_data = extract_features(URL)

    model = pickle.load(open('url_model.pkl', 'rb'))
    output=model.predict(user_url_data)

    url_type_mapping = {
        0: 'benign',
        1: 'defacement',
        2: 'phishing',
        3: 'malware'
    }

    result = url_type_mapping.get(output[0])

    print(f"Predicted URL Type: {result}")



def get_res():
    messagebox.showinfo(title='Malicious URL Result', message=result)

In [15]:
window = tk.Tk()
window.title("Malicious URL Detector")

style = ttk.Style()

style.theme_use('clam')

# Change the font and colors for buttons and labels
style.configure('TButton', font=('Arial', 12, 'bold'), foreground='white', background='blue')
style.configure('TLabel', font=('Arial', 14, 'bold'), foreground='black')
style.configure('TEntry', font=('Arial', 12))

url = tk.StringVar()

label1 = ttk.Label(window, text="URL:")
label1.grid(row=0, column=0, pady=10)

entry = ttk.Entry(window, width=80, textvariable=url)
entry.grid(row=0, column=1, pady=10)

submit_button = ttk.Button(window, text="Submit", command=action)
submit_button.grid(row=1, column=1, pady=20, padx=10)

predict_button = ttk.Button(window, text="Predict", command=make_predict)
predict_button.grid(row=2, column=0, pady=20, padx=10)

show_button = ttk.Button(window, text="Show", command=get_res)
show_button.grid(row=3, column=0, pady=20, padx=10)

window.geometry("700x250")

window.mainloop()

Predicted URL Type: phishing
Predicted URL Type: benign
